In [4]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import gaussian
import cv2

In [5]:
# Define a function to perform Non-Maximum Suppression (NMS)
def non_max_suppression(centers, threshold=10):
    if len(centers) == 0:
        return []

    # Convert list of points to a list of tuples for easier handling
    centers = [(x, y) for x, y in centers]

    # Initialize a list to store the filtered centers
    filtered_centers = []

    while len(centers) > 0:
        # Select the first center in the list
        best = centers[0]
        filtered_centers.append(best)

        # Calculate distances of remaining centers from the selected center
        distances = [np.sqrt((center[0] - best[0])**2 + (center[1] - best[1])**2) for center in centers[1:]]

        # Filter out centers that are within the threshold distance
        centers = [center for i, center in enumerate(centers[1:]) if distances[i] > threshold]
        
    return filtered_centers

In [6]:
import os

def process_images_in_folder(folder_path, output_folder):
    # Verificar si la ruta de salida existe, si no, crearla
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos en el directorio
    files = os.listdir(folder_path)
    
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            # Cargar la imagen
            image_path = os.path.join(folder_path, file)
            mask = cv2.imread(image_path, 0)

            if mask is not None:
                # Aplicar el algoritmo Canny para detección de bordes
                edges = cv2.Canny(mask, 30, 150)  # Los valores 30 y 150 son umbrales para el algoritmo Canny

                # Now, create a set of heatmaps that contains only a 1 at the coordinate
                img_h, img_w = edges.shape
                coords = [(111,111), (111,0), (0,55), (0,0), (0,111), (55,0), (55,111)]
                num_coords = len(coords)
                heatmaps = np.zeros((num_coords, img_h, img_w))


                for i in range(num_coords):
                    current_coord = coords[i]
                    heatmaps[i, current_coord[0], current_coord[1]] = 1

                # Now, apply a gaussian filter to each heatmap
                var = 20
                gaussian_heatmaps = gaussian(heatmaps, [0, var, var])

                # Now, use the contour as a mask of the heatmaps
                masked_heatmaps = np.zeros_like(gaussian_heatmaps)
                for i in range(num_coords):
                    masked_heatmaps[i] = edges * gaussian_heatmaps[i]
                    
                    # Renormalize so the values are between 0 and 1
                    max_val = np.max(masked_heatmaps[i])
                    masked_heatmaps[i] = masked_heatmaps[i] / max_val

                # Now, find the center of each heatmap
                landmark_centers = []
                for i in range(num_coords):
                    heatmap = (masked_heatmaps[i] * 255).astype(np.uint8)  # Convert to uint8

                    # Find the center of the heatmap
                    M = cv2.moments(heatmap)
                    if M["m00"] != 0:
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        landmark_centers.append((cX, cY))

                # Apply Non-Maximum Suppression
                threshold = 10
                filtered_landmark_centers = non_max_suppression(landmark_centers, threshold=threshold)
                while True:
                    if len(filtered_landmark_centers) == 7:
                        break
                    threshold = threshold - 1
                    filtered_landmark_centers = non_max_suppression(landmark_centers, threshold=threshold)

                # Generate a folder with the name of the image that will contain the landmarks
                new_folder = os.path.join(output_folder, (file.replace('mask', 'landmark')).replace('.jpg', ''))
                if not os.path.exists(new_folder):
                    os.makedirs(new_folder)

                # Generate an image with each landmark as a 1 channel image
                for i in range(len(filtered_landmark_centers)):
                    landmark = np.zeros_like(mask)
                    landmark[filtered_landmark_centers[i][1], filtered_landmark_centers[i][0]] = 255
                    cv2.imwrite(os.path.join(new_folder, '{}.png'.format(i)), landmark)

        else:
            print(f"No se pudo cargar la imagen {file} en la ruta especificada.")

# Llama a la función para procesar las imágenes en el directorio de entrada
for i in ['TRAIN', 'TEST', 'VAL']:
    input_folder_path = "../Mascaras/Mask/{}".format(i)
    output_folder_path = "./Landmarks/{}".format(i)
    process_images_in_folder(input_folder_path, output_folder_path)